<a href="https://colab.research.google.com/github/JockWang/tfboy/blob/master/Tensorflow2%E5%9F%BA%E7%A1%80%EF%BC%9A%E5%BC%A0%E9%87%8F%E3%80%81%E8%87%AA%E5%8A%A8%E6%B1%82%E5%AF%BC%E4%B8%8E%E4%BC%98%E5%8C%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

> 在 TensorFlow 1.X 版本中， 必须在导入 TensorFlow 库后调用tf.enable_eager_execution()函数以启用 Eager Execution 模式。在 TensorFlow 2.0 版本中，Eager Execution 模式为默认模式，无需额外调用tf.enable_eager_execution()函数（不过若要关闭 Eager Execution，则需调用tf.compat.v1.disable_eager_execution()函数）。

#### 张量

In [0]:
# 定义随机数
random_float = tf.random.uniform(shape=())
# 定义一个有两个元素的零向量
zero_vector = tf.zeros(shape=(2))
# 定义两个2×2的常量矩阵
A = tf.constant([[1., 2.],[3., 4.]])
B = tf.constant([[5., 6.],[7., 8.]])

In [4]:
# 张量的重要属性是形状、类型和值
print(A.shape)
print(A.dtype)
print(A.numpy())

(2, 2)
<dtype: 'float32'>
[[1. 2.]
 [3. 4.]]


**Noet**: Tensorflow的大多数API函数会根据输入的值自动推断张量中元素的类型（一般默认为tf.float32），当然*dtype*也可以在声明时定义。

#### 操作

In [5]:
C = tf.add(A, B)
print(C)
D = tf.matmul(A, B)
print(D)

tf.Tensor(
[[ 6.  8.]
 [10. 12.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


#### 自动求导

使用$tf.GradientTape()$计算函数$y(x)=x^2$在$x=3$时的导数：

In [7]:
import tensorflow as tf
#变量，变量和张量的一个重要区别是：变量默认能够被tensorflow的自动求导机制求导
x = tf.Variable(initial_value=3.) 
with tf.GradientTape() as tape:
  y = tf.square(x)
y_grad = tape.gradient(y, x)
print([y, y_grad])

[<tf.Tensor: id=43, shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: id=47, shape=(), dtype=float32, numpy=6.0>]


In [9]:
# 使用tf.GradientTape()对多元函数求偏导数
X = tf.constant([[1., 2.],[3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.],[2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
  L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

[62.5, array([[35.],
       [50.]], dtype=float32), 15.0]


#### 基础示例：线性回归

使用线性模型$y=ax+b$拟合房价数据，有$a$和$b$两个待求参数。

In [0]:
import numpy as np


In [0]:
X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min())/(X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min())/(y_raw.max() - y_raw.min())

In [12]:
# 使用NumPy
a, b = 0, 0

num_epoch = 10000
learning_rate = 1e-3
for e in range(num_epoch):
  y_pred = a * X + b
  grad_a, grad_b = (y_pred - y).dot(X), (y_pred - y).sum()

  # 更新
  a, b = a - learning_rate*grad_a, b - learning_rate*grad_b
print(a, b)

0.9763702027872221 0.057564988311377796


In [15]:
# Tensorflow实现
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)

variables = [a, b]
num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
for e in range(num_epoch):
  with tf.GradientTape() as tape:
    y_pred = a * X + b
    loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y))
  
  grads = tape.gradient(loss, variables)

  optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

print(a, b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.97637> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057565063>
